In [23]:
!pip install dnspython
!pip install pymongo[srv]

In [2]:
import pymongo
import pandas as pd
import re
import datetime as dt

In [81]:
client = pymongo.MongoClient("<Your DB Connection>")
db = client.Library
stu = db.student_details # using this collection for student details
book = db.book_details # book detailsb
dtl = db.details
conf = db.user_pass

In [59]:
def passwords(password):
    ids = 0
    l=0
    u=0
    num=0
    spl=0
    spl_char='!@#$%^()*/\][{}.,'
    password = input('Enter Your password \nYour password must have atleast 8 characters and one Special character, Numeric character and Upper Case character: ')
    if len(password)>=8:
        for i in password:
            if (i.islower()):
                l+=1
            elif (i.isupper()):
                u+=1
            elif (i.isdigit()):
                num+=1
            elif(i in spl_char):
                spl+=1
    else:
        print('please enter minimum 8 characters')
    if((l>=1)and(u>=1)and(num>=1)and(spl>=1) and l+u+num+spl==len(password)):
        final = {'password':password}
        tmp = conf.insert_one(final)
        ids = tmp.inserted_id
    return ids

In [60]:
# Registration
def Registration(selection):
    mailcheck = 0
    form= r'\b[a-z0-9./%+-_]+@[a-z]+.[a-z]{2,}\b'
    data = {
        'Name' : input('Enter Your Name: '),
        'RollNo' : 'GUVIDS' + input('Enter Your RollNo: '),
        'Dept' : input('Enter Your Dept: '),
        '_id' : int(input('Enter Your Mobile No: ')),
        'MailID' : input('Enter Your MailID: '),
        'Password' : passwords(None)
    }
    if (re.fullmatch(form,data['MailID'])):
        mailcheck = 1
    else:
        tmp = data['Password']
        conf.delete_one({'_id':tmp})
        print('Please Enter Valid MailID\nEx: Your mail ID should be like abc@abc.com')
    if len(str(data['_id'])) == 10 and mailcheck == 1:
        tmp = stu.insert_one(data)
        if tmp != None:
            a = stu.find({'_id':tmp.inserted_id},{'Password':0})
            df = pd.DataFrame(a)
            df.rename(columns = {'_id':'UserName'},inplace = True)
            display(df)
            print('You Have Successfully Registered !')

In [61]:
# login
def login(username):
    res = 0
    tmp = 0
    a = stu.find_one({'_id':username},{'_id':0,'Password':1})
    if a != None:
        tmp = a['Password']
        password = input('Please Enter Your password: ')
        b = conf.find_one({'_id' : tmp},{'_id':0,'password':1})
        if b['password'] == password:
            res = 1
        else:
            print('Wrong password Please try Again')
            login(username)
    else:
        print('No username available, Please Register')
    if res > 0:
        return True
    else:
        return False

In [62]:
def StudentFeatures(username):
    choice = int(input("You Have Successfully Logged in, please choose the Options to continue\n1. Search\n2. My View\n"))
    if choice == 1:
        val = input('Enter Book Title or Author or Genre: ')
        a = book.find({'$and' : [{'$or':[{'Title':val},{'Author':val},{'Genre':val}]},{'Issued':0}]},{'_id':0,'Issued':0})
        res = pd.DataFrame(a)
        display(res)
    elif choice == 2:
        a = dtl.find({'$and':[{'username' : username},{'Received':0}]},{'_id':0,'Title':1,'Issue_date':1,'Date_to_Return':1,'Penality':1})
        res = pd.DataFrame(a)
        display(res)

In [63]:
def BookEdit(selection,isbnno):
    if selection == 1:
        title = input('Enter Replacement Book Title: ')
        a = book.update_one({'_id':isbnno},{'$set':{'Title':title}})
        if a != None:
            print('Successfully Updated')
            temp = book.find({'_id':isbnno})
            df = pd.DataFrame(temp)
            df1 = df.replace({1:'Yes',0:'No'})
            display(df1)
        else:
            print('Not Updated, please try again')
    elif selection ==2:
        title = input('Enter Replacement Author: ')
        a = book.update_one({'_id':isbnno},{'$set':{'Author':title}})
        if a != None:
            print('Successfully Updated')
            temp = book.find({'_id':isbnno})
            df = pd.DataFrame(temp)
            df1 = df.replace({1:'Yes',0:'No'})
            display(df1)
        else:
            print('Not Updated, please try again')
    elif selection ==3:
        title = input('Enter Replacement Genre: ')
        a = book.update_one({'_id':isbnno},{'$set':{'Genre':title}})
        if a != None:
            print('Successfully Updated')
            temp = book.find({'_id':isbnno})
            df = pd.DataFrame(temp)
            df1 = df.replace({1:'Yes',0:'No'})
            display(df1)
        else:
            print('Not Updated, please try again')
    else:
        print('Please select correct option')

In [106]:
def IssueBook(username):
    a = stu.find_one({'_id':username})
    if a != None:
        cnt = dtl.count_documents({'$and' : [{'username': username},{'Received':0}]})
        if cnt < 5:
            isbn = int(input('Enter the ISBN number of the book:\n'))
            dts = book.find_one({'$and' : [{'_id':isbn},{'Issued':0}]})
            if dts != None:
                print(str(dts['_id']) + ' - '+dts['Title']+' - '+dts['Author'])
                resp = int(input('Press 1 to issue the above book details\n'))
                if resp == 1:
                    date = dt.datetime.strptime(input('Enter Date of Issue: Format : YYYY-MM-DD\n'),'%Y-%m-%d')
                    #date = dt.datetime.now()
                    data ={
                        'username' : username,
                        'Title' : dts['Title'],
                        'Author' : dts['Author'],
                        'ISBN' : isbn,
                        'Issue_date' : date,
                        'Date_to_Return' : date + dt.timedelta(days = 15),
                        'Returned_Date' : None,
                        'Penality' : 0,
                        'Penality_paid' : 0,
                        'Received' : 0
                    }
                    res = dtl.insert_one(data)
                    res1 = book.update_one({'_id':isbn},{'$set':{'Issued':1}})
                    if res != None and res1 != None:
                        print('Book Has Issued')
                    else:
                        print('Please Try Again')
                else:
                    print('Please select correct option')                
            else:
                print('No Book Available in this ISBN number, Please try again')
                IssueBook(username)
        else:
            print('User Has Exceeded the Book Limit')
    else:
        print('No such user Exist')

In [117]:
def ReceiveBook(username,isbn):
    a = stu.find_one({'_id':username})
    if a != None:
        #dts = dtl.find_one({'$and':[{'ISBN' : isbn},{'Received': 0}]},{'_id':0})
        dts = dtl.find_one({'ISBN' : isbn,'Received': 0,'username':username,'Penality_paid':0},{'_id':0})
        if dts != None:
            date = dt.datetime.strptime(input('Enter the Book received date Format:YYYY-MM-DD:\n'),'%Y-%m-%d')
            diff = date - dts['Issue_date']
            if diff.days > 0:
                tmp_dt = dtl.find_one({'ISBN':isbn},{'_id':0,'Date_to_Return':1})
                dif = date - tmp_dt['Date_to_Return']
                if dif.days <= 0:
                    res = dtl.update_one({'ISBN':isbn},{'$set':{'Returned_Date':date,'Received':1}})
                    res1 = book.update_one({'_id':isbn},{'$set':{'Issued' : 0}})
                    if res != None and res1 != None:
                        print('Successfully Returned')
                    else:
                        print('Error, please try again')
                else:
                    val = int(input('User has penality for this book press 1 to continue to collect penality\n'))
                    if val == 1:
                        PenalityCollect(username)
            else:
                print('Return Date should Greater than Issued Date')
                ReceiveBook(username,isbn)
        else:
            print('User has penalities or given book ISBN has not tagged to this user, please try again')
    else:
        print('Please check the username and try again')

In [118]:
def PenalityCollect(user):
    val = dtl.find({'$and':[{'username':user},{'Penality':{'$gt':0}}]},{'_id':0})
    if val != None:
        date = dt.datetime.now()
        df = pd.DataFrame(val)
        display(df)
        isbn = int(input('Enter the ISBN number of the book:\n'))
        amt = int(input('Enter the amount received:\n'))
        pen = dtl.find_one({'$and':[{'username':user,'ISBN':isbn},{'Penality' : {'$gt':0}}]})
        diff = amt - pen['Penality']
        if diff >= 0:
            upd = dtl.update_one({'$and':[{'username':user,'ISBN':isbn},{'Penality' : {'$gt':0}}]},{'$set':{'Penality':0,'Returned_Date': date,'Penality_paid':1,'Received':1}})
            upd1 = book.update_one({'_id':isbn},{'$set':{'Issued' : 0}})
            if upd != None and upd1 != None:
                print('Penality Paid and You need to pay remining RS:'+ str(diff))
        else:
            print('Kindly ask the user to give full penality amount')
            PenalityCollect(user)

In [111]:
def PenalityUpdate():
    a = dtl.find({'Received':0,'Penality_paid':0})
    for i in a:
        dif = dt.datetime.now() - i['Date_to_Return']
        days = dif.days
        if days > 0:
            pen = days * 20
            dtl.update_one({'_id':i['_id']},{'$set':{'Penality':pen}})

In [112]:
def adminFeature():
    opt = int(input('Choose your Option\n1. Add a Book\n2. Edit a Book\n3. Delete a Book\n4. Issue a Book\n5. Recieve a Book\n6. Penality Collector\n7. Issued Book View\n8. All Avilable Books\n'))
    if opt == 1:
        cnt = int(input('How many books do you need to add: '))
        while cnt > 0:
            book_dtl = {
                'Title' : input('Enter Book Title: '),
                'Author' : input('Enter Author: '),
                '_id' : int(input('Enter ISBN number: ')),
                'Genre' : input('Enter Genre of the Book: ')
            }
            if len(str(book_dtl['_id'])) == 5:
                res = book.insert_one(book_dtl)
                if res.acknowledged == True:
                    print('Book Added')
                    cnt -= 1
                else:
                    print('Book Not Added Please check once.')
            else:
                print('ISBN number must contains only 5 digits')
    elif opt == 2:
        isbn = int(input('Enter ISBN no of the book which you want to edit: '))
        tmp = book.find({'_id':isbn},{'_id':1,'Title':1,'Author':1,'Genre':1})
        if tmp != None:
            df = pd.DataFrame(tmp)
            display(df)
            sel = int(input('Select your option\n1. Edit Book Title\n2. Edit Author\n3. Edit Genre\n'))
            BookEdit(sel,isbn)
        else:
            print('Please check the ISBN number: ')
    elif opt == 3:
        isbn = int(input('Enter ISBN number of a book which you want to delete: \n'))
        res = book.delete_one({'_id':isbn})
        if res != None:
            print('Successfully Deleted')
        else:
            print('Not Deleted Please try again')
    elif opt == 4:
        usr = int(input('Enter the Username'))
        IssueBook(usr)
    elif opt == 5:
        usr = int(input('Enter username:\n'))
        ISBN = int(input('Enter Book ISBN number: '))
        ReceiveBook(usr,ISBN)
    elif opt == 6:
        usr = int(input('Enter username:\n'))
        PenalityCollect(usr)
    elif opt == 7:
        dtls = dtl.find({'Received':0},{'_id':0,'Returned_Date':0,'Penality_paid':0,'Received':0})
        pd.set_option('display.max_rows',2000)
        df = pd.DataFrame(dtls)
        display(df)
    elif opt == 8:
        books = book.find()
        pd.set_option('display.max_rows',2000)
        df = pd.DataFrame(books)
        df1 = df.replace({1:'Yes',0:'No'})
        display(df1)
    else:
        print('Please select correct option')
        adminFeature()

In [113]:
def Run():
    PenalityUpdate()
    role = int(input('Select your Role: \n1. Student\n2. Admin\n'))
    if role == 1:
        selection = int(input('1. Register\n2. Login'))
        if selection == 1:
            res = Registration(None)
        elif selection == 2:
            inp = int(input('Enter your Mobile no: '))
            status = login(inp)
            if status == True:
                StudentFeatures(inp)
            elif status == False:
                selection = int(input('Please press 1 if you want to Register'))
                if selection == 1:
                    Registration(None)
    elif role == 2:
        username = input('Enter Your username: ')
        password = input('Enter Your Password: ')
        if username == 'admin' and password == 'admin':
            print('You Have Successfully logged in')
            adminFeature()
        else:
            print('username or password incorrect, please try again')
    else:
        print('Please select appropriate role')
        Run()

In [ ]:
Run()